
# Enunciado del Problema

Dado un conjunto de datos de entrenamiento linealmente separables en dos dimensiones, implementaremos un modelo SVM para encontrar el hiperplano óptimo que maximice el margen entre dos clases. 
Los pasos incluyen:

1. Definir los puntos de entrenamiento y sus etiquetas.
2. Resolver el problema dual de optimización cuadrática para obtener los multiplicadores de Lagrange (α).
3. Identificar los vectores soporte (aquellos con α > 0).
4. Calcular el vector de pesos (θ) y el término independiente (b) del modelo.
5. Clasificar una nueva muestra [1, 1] usando el modelo entrenado.
6. Visualizar la frontera de decisión y los vectores soporte.

Los datos de entrenamiento son:

- **Clase +1 (y = +1)**: [2, 3], [3, 3], [2, 2]
- **Clase -1 (y = -1)**: [0, 1], [1, 0], [0, 0]

Este ejercicio utiliza la librería `cvxopt` para resolver el problema de optimización cuadrática y obtener los multiplicadores de Lagrange de forma precisa, cumpliendo con las condiciones de Karush-Kuhn-Tucker (KKT) para los puntos que están en el margen. El objetivo es obtener una clasificación óptima para datos linealmente separables y observar el comportamiento de los vectores soporte.


# Ejercicio SVM con Frontera de Decisión y Vectores Soporte


Este notebook resuelve un ejercicio de clasificación utilizando un SVM en Python. Los datos de ejemplo tienen dos clases linealmente separables y se utiliza `cvxopt` para resolver el problema de optimización cuadrática, obteniendo los multiplicadores de Lagrange, el vector de pesos, y el término independiente para la frontera de decisión. Luego, se clasifica una nueva muestra y se visualizan los resultados.


In [ ]:

# Importaciones necesarias
import numpy as np
import cvxopt
import cvxopt.solvers
import matplotlib.pyplot as plt


## Definición de datos

In [ ]:

# Definimos las muestras y sus etiquetas
X = np.array([[2, 3], [3, 3], [2, 2], [0, 1], [1, 0], [0, 0]])
y = np.array([1, 1, 1, -1, -1, -1])


## Configuración y resolución del problema dual usando `cvxopt`

In [ ]:

# Convertimos las etiquetas en formato columna para la optimización
Y = y[:, np.newaxis] * 1.0

# Calculamos la matriz de Gram (producto punto entre muestras) escalada por etiquetas
K = np.outer(y, y) * np.dot(X, X.T)

# Configuramos los parámetros del problema de optimización cuadrática
m, n = X.shape
P = cvxopt.matrix(K, tc='d')
q = cvxopt.matrix(-np.ones((m, 1)), tc='d')
G = cvxopt.matrix(-np.eye(m), tc='d')
h = cvxopt.matrix(np.zeros(m), tc='d')
A = cvxopt.matrix(y.reshape(1, -1).astype('double'))
b = cvxopt.matrix(0.0)

# Resolvemos el problema cuadrático usando cvxopt
solution = cvxopt.solvers.qp(P, q, G, h, A, b)

# Obtenemos los multiplicadores de Lagrange (alpha) de la solución
alphas = np.array(solution['x']).flatten()
alphas


## Identificación de vectores soporte

In [ ]:

# Filtramos los vectores soporte con un umbral para alphas mayores que cero
support_vector_indices = np.where(alphas > 1e-5)[0]
support_vectors = X[support_vector_indices]
support_labels = y[support_vector_indices]
support_alphas = alphas[support_vector_indices]
support_vectors, support_alphas


## Cálculo del vector de pesos y término independiente

In [ ]:

# Cálculo del vector de pesos theta*
theta_star = np.sum((support_alphas * support_labels)[:, np.newaxis] * support_vectors, axis=0)

# Cálculo del término independiente b* usando el primer vector soporte
b_star = support_labels[0] - np.dot(theta_star, support_vectors[0])
theta_star, b_star


## Clasificación de una nueva muestra [1, 1]

In [ ]:

# Clasificación de la nueva muestra
new_sample = np.array([1, 1])
decision_value = np.dot(theta_star, new_sample) + b_star
classification = 1 if decision_value > 0 else -1
classification


## Visualización de la frontera de decisión y vectores soporte

In [ ]:

# Generamos el espacio para la frontera de decisión
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
Z = (theta_star[0] * xx + theta_star[1] * yy + b_star)

# Realizamos el plot
plt.figure(figsize=(8, 6))

# Puntos de entrenamiento
plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], color='blue', marker='o', label='Clase +1')
plt.scatter(X[y == -1][:, 0], X[y == -1][:, 1], color='red', marker='s', label='Clase -1')

# Vectores soporte
plt.scatter(support_vectors[:, 0], support_vectors[:, 1], s=100, facecolors='none', edgecolors='black', label='Vectores Soporte')

# Frontera de decisión y márgenes
plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='green')  # Línea de decisión
plt.contour(xx, yy, Z, levels=[-1, 1], linestyles='--', colors='grey')  # Márgenes

# Configuraciones adicionales
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.title('SVM: Frontera de Decisión y Vectores Soporte')
plt.show()


## Verificación de la condición de KKT

In [ ]:

# Verificamos la condición de KKT
kkt_sum = np.sum(alphas * y)
kkt_sum
